In [1]:
import random
import torch
import torchmetrics

import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset



In [2]:
# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Example for RGB images
# ])

In [3]:
x = np.load("./data/balanced/input_data.npy")
labels = np.load("./data/balanced/label_data.npy", allow_pickle=True)

In [4]:
x_tensor = torch.from_numpy(x)
x_t = x_tensor.view(-1, 3, 49, 49)
inputs = x_t.float()


In [5]:
unique_labels = np.unique(labels)
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
print("Label Mapping:", label_mapping)
integer_labels = np.array([label_mapping[label] for label in labels])
targets = torch.from_numpy(integer_labels).long()


Label Mapping: {'Lg': 0, 'P': 1, 'Pg': 2, 'Pn': 3, 'S': 4, 'Sn': 5}


In [6]:
batch_size = 32
dataset = TensorDataset(inputs, targets)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [7]:
# x_min = inputs.min()
# x_max = inputs.max()
# print((x_min, x_max))

# inputs = 2 * (inputs - x_min) / (x_max - x_min) -1

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [9]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=2, padding=1)
        # self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(
            in_channels=32, out_channels=64, kernel_size=2, padding=1
        )
        # self.bn2 = nn.BatchNorm2d(64)  # Add BatchNorm after the second conv layer

        self.conv3 = nn.Conv2d(
            in_channels=64, out_channels=128, kernel_size=2, padding=1
        )

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        # self.bn3 = nn.BatchNorm2d(128)  # Add BatchNorm after the third conv layer

        # self.dropout = nn.Dropout(p=0.25)

        self.fc1 = nn.Linear(128 * 7 * 7, 256)
        self.bn_fc1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn_fc2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 6)

    def forward(self, x):
        x = self.pool(F.leaky_relu((self.conv1(x))))  # Conv1 -> BN -> ReLU -> Pool
        x = self.pool(F.tanh((self.conv2(x))))  # Conv2 -> BN -> Tanh -> Pool
        x = self.pool(F.tanh((self.conv3(x))))  # Conv3 -> BN -> Tanh -> Pool
        x = x.view(-1, 128 * 7 * 7)
        x = F.tanh(self.bn_fc1(self.fc1(x)))  # FC1 -> BN -> Tanh
        x = F.tanh(self.bn_fc2(self.fc2(x)))  # FC2 -> BN -> Tanh
        x = self.fc3(x)  # FC3 (no activation here because this is the final output)


        return x

In [10]:
model = CNN()
model = model.to(device=device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Rprop(model.parameters(), lr=0.0008)

# Warmup settings
initial_lr = 0.000008  # Starting learning rate for warmup
base_lr = 0.00008  # Target learning rate after warmup
warmup_steps = 10  # Number of warmup steps (epochs)
epochs = 50

# Learning rate scheduler after warmup
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# Inputs and targets to device
inputs = inputs.to(device=device)
targets = targets.to(device=device)

# Training loop with warmup steps and learning rate scheduler
for epoch in range(epochs):
    p_lr = ""
    # Warmup phase
    if epoch < warmup_steps:
        # Linear warmup
        lr = initial_lr + (base_lr - initial_lr) * (epoch / warmup_steps)
        for param_group in optimizer.param_groups:
            param_group["lr"] = lr
        # print(f"Warmup Epoch {epoch+1}/{warmup_steps}, Learning Rate: {lr:.8f}")
    else:
        # After warmup, allow the scheduler to control the learning rate
        scheduler.step()  # Step the scheduler after each epoch
        for param_group in optimizer.param_groups:
            lr = param_group["lr"]  # Get the current learning rate from the scheduler
    # print(f"Epoch {epoch+1}/{epochs}, Learning Rate: {lr:.8f}")

    # Training loop for each batch
    batch_loss = 0.0
    for batch_inputs, batch_targets in dataloader:
        batch_inputs = batch_inputs.to(device)
        batch_targets = batch_targets.to(device)

        optimizer.zero_grad()
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_targets)
        loss.backward()
        optimizer.step()

        batch_loss += loss.item()

    # Calculate average loss for the epoch
    avg_loss = batch_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{epochs}, Cross Entropy Loss: {avg_loss:.4f},Learningrate: {lr: 0.8f}")

Epoch 1/50, Cross Entropy Loss: 1.7089,Learningrate:  0.00000800
Epoch 2/50, Cross Entropy Loss: 1.6344,Learningrate:  0.00001520
Epoch 3/50, Cross Entropy Loss: 1.5959,Learningrate:  0.00002240
Epoch 4/50, Cross Entropy Loss: 1.5635,Learningrate:  0.00002960
Epoch 5/50, Cross Entropy Loss: 1.5374,Learningrate:  0.00003680
Epoch 6/50, Cross Entropy Loss: 1.5133,Learningrate:  0.00004400
Epoch 7/50, Cross Entropy Loss: 1.4940,Learningrate:  0.00005120
Epoch 8/50, Cross Entropy Loss: 1.4790,Learningrate:  0.00005840
Epoch 9/50, Cross Entropy Loss: 1.4634,Learningrate:  0.00006560
Epoch 10/50, Cross Entropy Loss: 1.4466,Learningrate:  0.00007280
Epoch 11/50, Cross Entropy Loss: 1.4324,Learningrate:  0.00007280
Epoch 12/50, Cross Entropy Loss: 1.4198,Learningrate:  0.00007280
Epoch 13/50, Cross Entropy Loss: 1.4065,Learningrate:  0.00007280
Epoch 14/50, Cross Entropy Loss: 1.3957,Learningrate:  0.00007280
Epoch 15/50, Cross Entropy Loss: 1.3863,Learningrate:  0.00007280
Epoch 16/50, Cross 

In [11]:
x = np.load("./data/balanced/test_data.npy")
labels = np.load("./data/balanced/test_label_data.npy", allow_pickle=True)

x_tensor = torch.from_numpy(x)
x_t = x_tensor.view(-1, 3, 49, 49)
inputs = x_t.float()
inputs = inputs.to(device)

unique_labels = np.unique(labels)
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
print("Label Mapping:", label_mapping)
integer_labels = np.array([label_mapping[label] for label in labels])
targets = torch.from_numpy(integer_labels)
targets = targets.long()
targets = targets.to(device=device)

Label Mapping: {'Lg': 0, 'P': 1, 'Pg': 2, 'Pn': 3, 'S': 4, 'Sn': 5}


In [12]:
y_pred = model(inputs)
probabilities = F.softmax(y_pred, dim=1)
predicted_class = torch.argmax(probabilities, dim=1)

predicted_class = predicted_class.to(device=device)
print(targets)

tensor([4, 1, 0,  ..., 5, 0, 5], device='cuda:0')


In [13]:
accuracy_metric  = torchmetrics.Accuracy(task="multiclass", num_classes=6).to(device=device)
accuracy = accuracy_metric(predicted_class, targets)
accuracy

tensor(0.4488, device='cuda:0')

In [14]:
test_data = np.load("./data/balanced/test_data.npy")
test_labels = np.load("./data/balanced/test_label_data.npy", allow_pickle=True)

test_data_tensor = torch.from_numpy(test_data)
test_data_tensor = test_data_tensor.view(-1, 3, 49, 49)  # Reshape to [batch_size, 3, 49, 49]
test_inputs = test_data_tensor.float()
test_inputs = test_inputs.to(device)

unique_labels = np.unique(test_labels)
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
print("Label Mapping:", label_mapping)
integer_test_labels = np.array([label_mapping[label] for label in test_labels])

test_targets = torch.from_numpy(integer_test_labels).long()
test_targets = test_targets.to(device)

batch_size = 32  # Define the batch size
test_dataset = TensorDataset(test_inputs, test_targets)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


accuracy = torchmetrics.Accuracy(task="multiclass", num_classes=len(unique_labels)).to(device)
model.eval()

with torch.no_grad():
    for batch_test_inputs, batch_test_targets in test_dataloader:
        outputs = model(batch_test_inputs)

        predictions = torch.argmax(outputs, dim=1)
        accuracy.update(predictions, batch_test_targets)

final_accuracy = accuracy.compute().item()
print(f"Accuracy on the test data: {final_accuracy * 100:.2f}%")

accuracy.reset()

Label Mapping: {'Lg': 0, 'P': 1, 'Pg': 2, 'Pn': 3, 'S': 4, 'Sn': 5}
Accuracy on the test data: 44.57%
